画像を、犬か猫かの２択（「分からない」無し）で分類するAIを作りたい！

# 事前準備

## ライブラリのインポート

In [ ]:
!pip install icrawler

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch.utils.data
from torchsummary import summary
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import random
import tqdm, time

from PIL import Image
import glob
from sklearn.metrics import confusion_matrix
import seaborn as sn  # 可視化に関するライブラリです。
import pandas as pd

## 定数の定義

In [ ]:
classes = ["dog", "cat"]
num_classes = len(classes)
scraping_max = 200# スクレイピングで集める画像の最大枚数
image_size = 64   # 画像サイズ（vgg11：224, 自前NN：64）
num_testdata = 25 # テストデータの数
BATCH_SIZE = 175  # バッチサイズ
EPOCHS = 50       # エポック数
NPY_FILE_NAME = "dog_cat.npy" # 画像データの保存ファイル名

# 使用デバイス
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## 画像データのスクレイピング

参考サイト：[AI Academy | Deep Learningで犬・猫を分類してみよう](https://aiacademy.jp/texts/show/?id=164)

### 画像を集める

In [ ]:
# データ収集
from icrawler.builtin import BingImageCrawler

# 「猫」で画像収集
crawler = BingImageCrawler(downloader_threads=4, storage={"root_dir": "cat"})
crawler.crawl(keyword="猫", max_num=scraping_max)

# 「犬」で画像収集
crawler = BingImageCrawler(downloader_threads=4, storage={"root_dir": "dog"})
crawler.crawl(keyword="犬", max_num=scraping_max)

### 画像をndarrayに変換してファイルに保存する

In [ ]:
# 画像の一部切り取りプロセス
train_process = transforms.Compose([
  transforms.Resize(image_size*4//3),
  transforms.RandomCrop(image_size),
])

# 画像データ変換処理
X_train = []
X_test = []
Y_train = []
Y_test = []
for index, classlabel in enumerate(classes):
  # フォルダにある特定の種類の動物の画像ファイルを読み込む
  photos_dir = "./" + classlabel
  files = glob.glob(photos_dir + "/*.jpg")
  for i, file in enumerate(files[:140]):
    # 画像データを１つずつ読み込み、サイズ等の調整
    image = Image.open(file)
    image = image.convert("RGB")

    if i < num_testdata:
      # テストデータとして保存する
      image = image.resize((image_size, image_size))
      data = np.asarray(image)
      X_test.append(data)
      Y_test.append(index)
    else:
      # 学習用データとして保存する
      image2 = image.copy()
      width, height = image.size
      for i in range(2):
        image = train_process(image)

        # 角度を少しずつずらしたものをそれぞれ保存する
        for angle in range(-20, 20, 10):
          img_r = image.rotate(angle)
          data = np.asarray(img_r)
          X_train.append(data)
          Y_train.append(index)
          img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
          data = np.asarray(img_trains)
          X_train.append(data)
          Y_train.append(index)

# 配列をndarray型に直し、npyファイル（dog_cat.npy）として保存する
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
xy = (X_train, X_test, Y_train, Y_test)
np.save("./" + NPY_FILE_NAME, xy)

## データセットの準備

### NPYファイルを読み込む

In [ ]:
# 乱数ジェネレータの生成
rng = np.random.default_rng()

# npyファイルを読み込む
def load_data():
  X_train, X_test, Y_train, Y_test = np.load("./" + NPY_FILE_NAME, allow_pickle=True)

  # 入力データの正規化
  X_train = torch.from_numpy(X_train.astype(np.float32) * 2 / 255 - 1)
  X_test = torch.from_numpy(X_test.astype(np.float32) * 2 / 255 - 1)
  # 出力データをワンホットベクトルに変換
  Y_train = torch.from_numpy(Y_train)
  Y_test = torch.from_numpy(Y_test)
  return X_train, X_test, Y_train, Y_test

sample = load_data()

### データローダーの準備

In [ ]:
# データセットの作成
train_dataset = torch.utils.data.TensorDataset(sample[0], sample[2])#.to(torch.float32))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(sample[1], sample[3])#.to(torch.float32))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 形状の計算
input_shape = (image_size, image_size, 3)
output_shape = num_classes

## 画像データ例の表示

In [ ]:
# テストデータを表示
pos = 1
index = random.randint(0, sample[1].shape[0] / 2)
i = index

plt.figure(figsize=(16, 5))

for img in sample[1][index:index + 30]:
    plt.subplot(3, 10, pos)
    plt.imshow(img * 0.5 + 0.5)
    plt.axis('off')
    plt.title(classes[torch.argmax(sample[3][i])])
    pos += 1
    i += 1

plt.show()

In [ ]:
# 学習用データを表示
pos = 1
index = random.randint(0, sample[0].shape[0] / 2)
i = index

plt.figure(figsize=(16, 5))

for img in sample[0][index:index + 30]:
    plt.subplot(3, 10, pos)
    plt.imshow(img * 0.5 + 0.5)
    plt.axis('off')
    plt.title(classes[torch.argmax(sample[2][i])])
    pos += 1
    i += 1

plt.show()

# 画像データを学習

## NNモデルを準備

### 学習済モデルから用意する場合（ファインチューニング）
(無料版Colabのメモリ制限的に上手く学習するのは厳しかった)

In [ ]:
# 学習済みの重みを使用
use_pretrained = True

# vgg11モデルをロード（入力に用いる画像は224×224を使用）
model = models.vgg11(pretrained=use_pretrained)

# 変更前のモデルを出力
print("変更前モデル")
print(model)

# 最後の層を変更
model.classifier[6] = nn.Linear(4096, 2)
softmax = nn.Softmax(dim=1)

# 変更後のモデルを出力
print("変更後モデルの線形層")
print(model.classifier)

model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters())

summary(model, (3, image_size, image_size))


### NNモデルを自前で用意する場合

In [ ]:
# 入力に用いる画像サイズは64×64を想定
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 入力のチャネル、出力のチャネル（＝フィルター数）、フィルタのサイズ（3 * 3）
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)

        # 64 -> 32 -> 16 -> 8 -> 4
        # 128 * 4 * 4 = 2048
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))

        x = self.fc2(x)
        x = self.softmax(x)
        return x


model = Net()
model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters())
summary(model, (3, image_size, image_size))

softmax = nn.Softmax(dim=1)

print('入力のサイズ:', input_shape)
print('出力のサイズ:', output_shape)

## モデルの学習

In [ ]:
# 損失と正解率の計算関数
def calc_loss_and_accuracy(loader_list):
    acc = 0
    cnt = 0
    loss = 0
    for (x, y) in tqdm.tqdm(loader_list, "evaluating... ", position=0, leave=True):
        x = x.to(device)
        y = y.to(device)
        x = torch.reshape(x, (-1, 3, image_size, image_size))
        y = torch.reshape(y, (-1,))
        
        pred = model(x)
        loss += criterion(pred, y).item()
        pred = torch.argmax(pred, dim=1)
        for o, t in zip(pred, y):
            if o == t:
                acc += 1
            cnt += 1
    return loss / len(loader_list), acc / cnt

In [ ]:
# 学習
def train(epochs):
    train_losses, train_accs, test_losses, test_accs = [], [], [], []
    train_loader_list = list(train_loader)
    test_loader_list = list(test_loader)
    for epoch in range(epochs):
        for (x, y) in tqdm.tqdm(train_loader_list, f"[{epoch + 1}/{epochs}] training... ", position=0, leave=True):
            x = torch.reshape(x, (-1, 3, image_size, image_size))
            y = torch.reshape(y, (-1,))#2))
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y) * 10
            loss.backward()
            optimizer.step()
        #print(pred[:5])

        train_loss, train_acc = calc_loss_and_accuracy(train_loader_list)
        test_loss, test_acc = calc_loss_and_accuracy(test_loader_list)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        print(
            f'\n[*] train_Loss = {train_loss} | train_Accuracy = {train_acc} | test_Loss = {test_loss}| test_Accuracy = {test_acc}')
    return train_losses, train_accs, test_losses, test_accs

st = time.time()
train_losses, train_accs, test_losses, test_accs = train(EPOCHS)
elapsed_time = time.time() - st


## 学習結果の確認

In [ ]:
print('Train loss:', train_losses[len(train_losses) - 1])
print('Test loss:', test_losses[len(test_losses) - 1])

print('Train accuracy:', train_accs[len(train_accs) - 1])
print('Test accuracy:', test_accs[len(test_accs) - 1])

print('実行時間(秒) : ', elapsed_time)

## 学習過程をグラフで表示

In [ ]:
plt.plot(train_accs)
plt.plot(test_accs)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(train_losses)
plt.plot(test_losses)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## 学習モデルのテスト

In [ ]:
# 学習、テストで用いていない画像データで試す
for index, classlabel in enumerate(classes):
  # 描画用意
  plt.figure(figsize=(16, 5))

  # フォルダにある特定の種類の動物の画像ファイルを読み込む
  photos_dir = "./" + classlabel
  files = glob.glob(photos_dir + "/*.jpg")
  for i, file in enumerate(files[140:145]):
    # 画像データを１つずつ読み込み、サイズ等調整
    image = Image.open(file)
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    X_check = np.asarray(data)
    X_check = torch.from_numpy(X_check.astype(np.float32) * 2 / 255 - 1)

    # 画像データの描画
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_check * 0.5 + 0.5)
    plt.axis('off')
    plt.title(classlabel)

    # 予測
    X_check = torch.reshape(X_check, (-1, 3, image_size, image_size))
    X_check = X_check.to(device)
    optimizer.zero_grad()
    pred = model(X_check)
    pred = softmax(pred)
    print(classlabel + str(i) + f"の予測結果: 犬={pred[0, 0]*100:.3f}%, 猫={pred[0, 1]*100:.3f}% -> 結果: {classes[torch.argmax(pred[0])]}")

  plt.show()


## モデルの重みの保存・読み込み

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model.load_state_dict(torch.load('model.pth', map_location=device))